In [6]:
from interpreter import interpreter
from interpreter.utils import take_screenshot
import os

In [3]:
SYSTEM_PROMPT = """
You are an expert GUI automation agent that performs tasks in web browsers. 
You analyze screenshots and user queries to determine the exact sequence of actions needed.
You can only interact with the GUI using keyboard actions. So use keyboard shortcuts to navigate and perform tasks.
When typing mention where you are typing (e.g., address bar, search box, form field).
Process Flow:
    1. Analyze the current screenshot and query to understand the task
    2. Reason step-by-step about required actions to take based on the current screen state.
    3. After reasoning all the steps, you will describe and execute one action by outputting a JSON object with the required action.
    4. Wait for a new screenshot to be provided. Use this screenshot to check if the action was successful before continuing with the next action. If the action was not successful, adjust your approach based on the new screenshot.
    5. Output plain text when keyboard actions are not required.
    6. Repeat steps 1-5 until the task is complete.
    7. Once the task is complete based on the current screenshot, explain what you did and why, then output {"tool": "task_complete"} to indicate the task is done.
Output Format:
    1. For reasoning steps: Output plain text describing your thought process based on the current screenshot.
    2. For keyboard actions: Output description of the action being taken based on the current screenshot and the python code to execute the action based on the current screenshot.
program: 
    1. Use the pyautogui library to perform keyboard actions. 
    2. Import pyautogui at the start of your program.
    3. Use pyautogui.hotkey() for keyboard shortcuts. 
    4. When typing go to the appropriate field first (e.g., address bar) then use pyautogui.write() to type.
Important Notes:
    1. Always analyze the current screenshot before taking any action.
    2. If an action does not lead to the expected result of the task, adjust your approach based on the new screenshot."""

In [9]:
# Query to ask the assistant
query = "Create an excel sheet of 10 students 5 subject and give them the average score and percentage in new column."

# Safety: default to dry-run. Set this to True only when you are ready
# for the notebook to control your keyboard/mouse.
EXECUTE_GUI = True  # set to True to allow pyautogui actions

# Take a screenshot (uses interpreter.utils.take_screenshot if available,
# otherwise falls back to the capture_screenshot defined elsewhere in this notebook).
screenshot_path = None
try:
    screenshot_path = take_screenshot()
except Exception as e:
    print('take_screenshot() failed, trying capture_screenshot fallback:', e)

# Build the message with optional image
message_with_image = [{"role": "user", "type": "message", "content": query}]
if screenshot_path:
    message_with_image.append({"role": "user", "type": "image", "format": "path", "content": screenshot_path})

# Send to interpreter and capture the response
response = interpreter.chat(message_with_image)

Exiting...

SystemExit: 0

c:\Users\sathish\miniconda3\envs\py3.10\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To 
exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)